# Example on line integrals of a vector field

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from glob import glob
from matplotlib import cm
import pyrotor

## Define setting

### Define cost function

In [2]:
# Model - \int_0^1 F(y(t)) y'(t) dt with F(y) = (0, y_1)
# Quadratic part
q = np.array([[0,0,0,-1],
              [0,0,0,0],
              [0,0,0,0],
              [-1,0,0,0]])
# Linear part
w = np.array([0,0,0,0])

# Constant part
c = 0

quadratic_model = [c, w, q]

### Define initial and final states

In [3]:
endpoints = {'x1': {'start': .111,
                    'end': .912,
                    'delta': 0.0001},
             'x2': {'start': .926,
                    'end': .211,
                    'delta': 0.0001}}

### Define independent variable (time)

In [4]:
independent_variable = {'start': .1,
                        'end': .9,
                        'frequency': .01}
# Compute number of evaluation points
delta_time = independent_variable['end'] - independent_variable['start']
delta_time /= independent_variable['frequency']
independent_variable['points_nb'] = int(delta_time) + 1

### Define constraints

In [5]:
# Constraints
# x1 > 0
def f1(data):
    x1 = data["x1"].values
    return x1

# x1 < 1
def f2(data):
    x1 = data["x1"].values
    return 1 - x1

# x2 > 0
def f3(data):
    x2 = data["x2"].values
    return x2

# x2 < 1
def f4(data):
    x2 = data["x2"].values
    return 1 - x2

constraints = [f1, f2, f3, f4]

### Define functional basis

In [6]:
# Basis name
basis = 'legendre'
# Dimension for each variable
basis_dimension = {'x1': 4,
                   'x2': 6}

### Import reference trajectories

In [7]:
reference_trajectories = pyrotor.datasets.load_toy_dataset('example_1')

/home/arthur/miniconda3/lib/python3.8/site-packages/pyrotor/toy_dataset/example_1/*.csv


## Optimize

### Create PyRotor class

In [8]:
sigma = np.eye(10)

mr_pyrotor = pyrotor.Pyrotor(quadratic_model, 
                             reference_trajectories, 
                             endpoints, 
                             constraints, 
                             basis, 
                             basis_dimension, 
                             independent_variable, 
                             n_best_trajectory_to_use=5,
                             opti_factor=1,
                             #sigma=sigma,
                             derivative=True,
                             use_quadratic_programming=False,
                             verbose=True)

### Execute PyRotor solver

In [ ]:
mr_pyrotor.compute_optimal_trajectory()
print(mr_pyrotor.trajectory)

### Compute savings

In [ ]:
mr_pyrotor.compute_gains()

In [ ]:
mr_pyrotor.compute_relative_gains()

## Plot

In [ ]:
X = np.linspace(independent_variable['start'],
                independent_variable['end'],
                independent_variable['points_nb'])

positions_x, positions_y = np.meshgrid(np.arange(0,1,.1), np.arange(0,1,.1))
arrays_y = .5 * positions_x
arrays_x = np.zeros_like(arrays_y)

cost = - mr_pyrotor.cost_by_time

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 20))
ax1.plot(X, mr_pyrotor.trajectory['x1'])
ax1.plot(X, mr_pyrotor.reference_trajectories[0]['x1'], linestyle=':')
ax1.set_xlabel('$t$')
ax1.set_ylabel('$x_1(t)$')
ax2.plot(X, mr_pyrotor.trajectory['x2'])
ax2.plot(X, mr_pyrotor.reference_trajectories[0]['x2'], linestyle=':')
ax2.set_xlabel('$t$')
ax2.set_ylabel('$x_2(t)$')
ax3.plot(mr_pyrotor.trajectory['x1'], mr_pyrotor.trajectory['x2'], c='b', alpha=.5,
         label='Optimized trajectory')
im = ax3.scatter(mr_pyrotor.trajectory['x1'], mr_pyrotor.trajectory['x2'], c=cost)
for trajectory in mr_pyrotor.reference_trajectories:
    ax3.plot(trajectory['x1'], trajectory['x2'], linestyle=":", label='_nolegend_')
ax3.quiver(positions_x, positions_y, arrays_x, arrays_y, color='r', width=.002, alpha=.3)
fig.colorbar(im, ax=ax3)
ax3.set_xlabel('$x_1$')
ax3.set_ylabel('$x_2$')
ax3.set_xlim(left=0, right=1)
ax3.set_ylim(bottom=0, top=1)
ax3.legend()
plt.tight_layout()